---
Tom Curran

Problem Set \#5

MACS 30100

Monday February 19, 2018

---

**Multinomial logistic regression and cross validation (6 points)**. For this problem, you will estimate the probability that a given wine comes from a given cultivar. The data in the file strongdrink.txt (taken from the UCI Machine Learning Repository) are the results of a chemical analysis of 176 Italian wines from three known cultivars (a cultivar is a group of grapes selected for desirable characteristics that can be maintained by propagation). The chemical analysis determined the quantities of the following 13 different constituents (the last 13 variables):

Use a multinomial logistic regression model of the following form with the following linear predictor ηj for j = 1, 2 (the baseline class is j = 3).


$$ Pr ( culitvar_{i} = j\ | X \beta_{j} )$$

Estimate the model on a 75% sample training set using the following com- mand. Report your two sets of estimated coefficients for j = 1 and j = 2. Report your error rates (1 - precision) on the test set using the code below. Which category of cultivar is the model best at predicting? Is the most accurately predicted category the one with the most observations?

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn import metrics 
from sklearn.metrics import classification_report, mean_squared_error
from pylab import rcParams
df = pd.read_csv("strongdrink.txt")

/Users/thomascurran/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df.columns

test = df[['cultivar',
          'alco',
          'malic',
          'tot_phen',
          'color_int']]
test['constant'] = 1
test.head()


/Users/thomascurran/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,cultivar,alco,malic,tot_phen,color_int,constant
0,1,14.23,1.71,2.80,5.64,1
1,1,13.20,1.78,2.65,4.38,1
2,1,13.16,2.36,2.80,5.68,1
3,1,14.37,1.95,3.85,7.80,1
4,1,13.24,2.59,2.80,4.32,1


In [3]:
test.isnull().sum()

cultivar     0
alco         0
malic        0
tot_phen     0
color_int    0
constant     0
dtype: int64

In [25]:
#exog
x = test[['alco',
         'malic',
         'tot_phen',
         'color_int']]
x.insert(0,'constant',1)
#endog
y = test['cultivar']

In [26]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size= .25, random_state = 20)

In [27]:
multilogreg = LogisticRegression(multi_class = 'multinomial',solver = 'newton-cg')

results = multilogreg.fit(x_train, y_train)

coefficients = results.coef_

yhat = multilogreg.predict(x_test)

confusion_matrix = metrics.confusion_matrix(y_test, yhat)

classification = classification_report(y_test, yhat)

model_mean_squared_error = mean_squared_error(y_test, yhat)

In [28]:
print("Coefficients when j = 1:", coefficients[0])
print("Intercept when j = 1:", results.intercept_[0])
print("------------------------------------------------")
print("Coefficients when j = 2:", coefficients[1])
print("Intercept when j = 2:", results.intercept_[1])

Coefficients when j = 1: [ -8.45998030e-06   1.70038994e+00  -2.65604001e-01   1.22389318e+00
   2.27585993e-02]
Intercept when j = 1: -24.0108147994
------------------------------------------------
Coefficients when j = 2: [ -1.76923782e-05  -1.46805313e+00  -3.33053748e-01   6.64013944e-01
  -9.22712974e-01]
Intercept when j = 2: 22.8025761029


In [29]:
print(confusion_matrix)

[[13  0  0]
 [ 2 19  0]
 [ 0  0 10]]


In [30]:
print(classification)

             precision    recall  f1-score   support

          1       0.87      1.00      0.93        13
          2       1.00      0.90      0.95        21
          3       1.00      1.00      1.00        10

avg / total       0.96      0.95      0.96        44



In [31]:
print(multilogreg.intercept_)

[-24.0108148  22.8025761   1.2082387]


In [32]:
print(multilogreg.classes_)

[1 2 3]


In [33]:
print(model_mean_squared_error)

0.0454545454545


In [34]:
#number of obs per cultivar
test.groupby(['cultivar'])['cultivar'].count()

cultivar
1    59
2    71
3    46
Name: cultivar, dtype: int64

***

Error Rates:

$$ Cultivar_1 = 1 - .87 = .13 $$
$$ Cultivar_2 = 1 - 1 = 0 $$
$$ Cultivar_3 = 1 - 1 = 0 $$

The model is best at predicting cultivar \#3 since its error rate was 0%, while Cultivar 2 also had a 0% error rate, it had a lower recall value, and Cultivar 1 was the worst predicted since there was a 13% error rate. 

Based on the model, the most acurately predicted cultivar is not the one with the most observations. 
***

(b) Perform a leave-one-out cross validation (LOOCV) with the model from part (a). Report your error rates (1 - precision) for each category? How do your error rates compare to those from part (a)? Report your LOOCV estimate for the test MSE as the average MSE, where yi is the left out observation from each test set.

In [35]:
x_loo = np.array(x) #copy of 1a model

y_loo = np.array(y) #copy of 1a model

n_loo = x_loo.shape[0] #get the length of the number of obs

loo = LeaveOneOut() #initialize the LOO function

loo.get_n_splits(x_loo) #number of times that LOO is going to split the column

mse_loo = np.zeros(n_loo) #create a n number of zeroes to form a matrix

In [36]:
ytest_val = np.zeros(n_loo)
ypred_val = np.zeros(n_loo)


for train_index, test_index in loo.split(x_loo):
    x_train_loo, x_test_loo = x_loo[train_index], x_loo[test_index]
    y_train_loo, y_test_loo = y_loo[train_index], y_loo[test_index]
    
    ytest_val[test_index] = y_test_loo
    
    loo_logreg = LogisticRegression(multi_class = 'multinomial', solver = 'newton-cg')
    
    loo_logreg.fit(x_train_loo, y_train_loo)
    
    yhat_loo = loo_logreg.predict(x_test_loo)
    
    ypred_val[test_index] = yhat_loo

    mse_loo[test_index] = (y_test_loo - yhat_loo)**2
    
    #print("Mean Squared Error for test set number ", test_index,": ", mse_loo[test_index])
    
mse_loo_mean = mse_loo.mean()

mse_loo_std = mse_loo.std()
print("-------------------------------------")
print("Mean Squared Error for LOO Cross Validation: ", mse_loo_mean)
print("Standard Deviation of Mean Squared Error for LOO Cross Validation: ", mse_loo_std)

-------------------------------------
Mean Squared Error for LOO Cross Validation:  0.0965909090909
Standard Deviation of Mean Squared Error for LOO Cross Validation:  0.394262505894


In [37]:
print(classification_report(ytest_val, ypred_val))

             precision    recall  f1-score   support

        1.0       0.90      0.93      0.92        59
        2.0       0.91      0.90      0.91        71
        3.0       0.96      0.93      0.95        46

avg / total       0.92      0.92      0.92       176



***

Error Rates:

$$ Cultivar_1 = 1 - .90 = .10 $$
$$ Cultivar_2 = 1 - .91 = .9 $$
$$ Cultivar_3 = 1 - .96 = .4 $$

This model is worse at predicting cultivar than the model produced in part a

***

1c) K-Fold Cross Validation:

In [44]:
k = 4

xvals = np.array(x)

yvals = np.array(y)

kf = KFold(n_splits = k, shuffle = True, random_state = 10)

kf.get_n_splits(xvals)

mse_k = np.zeros(k)

In [45]:
k_index = int(0)
k_ytest_val = np.zeros(n_loo)
k_ypred_val = np.zeros(n_loo)

for train_index, test_index in kf.split(xvals):
    
    x_train_k, x_test_k = xvals[train_index], xvals[test_index]
    y_train_k, y_test_k = yvals[train_index], yvals[test_index]
    
    multilogreg_k = LogisticRegression()
    
    multilogreg_k.fit(x_train_k, y_train_k)
    
    yhat_k = multilogreg_k.predict(x_test_k)
    
    k_ytest_val[test_index] = y_test_k
    
    k_ypred_val[test_index] = yhat_k
        
    mse_k[k_index] = ((y_test_k - yhat_k)**2).mean()

    print("k index: ", k_index,"|", "MSE: ", mse_k[k_index])
    
    k_index += 1

k index:  0 | MSE:  0.363636363636
k index:  1 | MSE:  0.227272727273
k index:  2 | MSE:  0.181818181818
k index:  3 | MSE:  0.0454545454545


In [46]:
mse_k_mean = mse_k.mean()
mse_k_std = mse_k.std()
print("test esimate for MSE k-fold: ", mse_k_mean," | test estimate standard error: ", mse_k_std)

test esimate for MSE k-fold:  0.204545454545  | test estimate standard error:  0.113636363636


In [47]:
print(classification_report(k_ytest_val, k_ypred_val))

             precision    recall  f1-score   support

        1.0       0.81      0.75      0.78        59
        2.0       0.81      0.87      0.84        71
        3.0       0.96      0.93      0.95        46

avg / total       0.85      0.85      0.85       176



***

Error Rates:

$$ Cultivar_1 = 1 - .81 = .19 $$
$$ Cultivar_2 = 1 - .81 = .19 $$
$$ Cultivar_3 = 1 - .96 = .4 $$

This model is worse at predicting cultivar than the model produced in part a and in part b

***